# 1. 使用langchain

In [ ]:
!pip install langchain

# 2. 导入第三方库

In [4]:
import os
import sys
import tarfile

from erniebot_agent.agents import RetrievalAgent
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import WholeMemory
from erniebot_agent.retrieval import BaizhongSearch
from erniebot_agent.tools.baizhong_tool import BaizhongSearchTool
from erniebot_agent.tools.langchain_retrieval_tool import LangChainRetrievalTool
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import SpacyTextSplitter
from erniebot_agent.utils.common import download_file

# 3. 预处理

## 3.1 下载数据集

In [5]:
if not os.path.exists("construction_regulations"):
    file_path = "construction_regulations.tar"
    download_file(
        url="https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar",
        save_path=file_path,
    )

    files = tarfile.open(file_path, "r:*")
    file_dir = os.path.dirname(file_path)
    files.extractall(file_dir, files.getmembers())
# ! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
# ! tar xvf construction_regulations.tar

## 3.2 构建知识库

In [7]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada")

/root/anaconda3/envs/py39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [8]:
faiss_name = "faiss_index"
if os.path.exists(faiss_name):
    db = FAISS.load_local(faiss_name, embeddings)
else:
    loader = PyPDFDirectoryLoader("construction_regulations")
    documents = loader.load()
    text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=320, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_name)

# 4. 基于复杂Query分解的RetrievalAgent
## 4.1 复杂任务规划

In [12]:
search_tool = LangChainRetrievalTool(db=db, threshold=0.0)
llm = ERNIEBot(model="ernie-3.5", api_type="aistudio")
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=False,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [13]:
query = "住房和城乡建设部发布城市设计管理办法有哪些？"
response = await agent.run(query)
response

## 4.2 使用compressor压缩子query检索结果

In [17]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [18]:
query = "住房和城乡建设部发布城市设计管理办法有哪些？"
response = await agent.run(query)
response

AgentResponse(text='住房和城乡建设部发布的城市设计管理办法包括以下内容：\n\n1. 第一条：为提高城市建设水平，塑造城市风貌特色，推进城市设计工作，完善城市规划建设管理，依据《中华人民共和国城乡规划法》等法律法规，制定本办法。\n2. 第二条：城市、县人民政府所在地建制镇开展城市设计管理工作，适用本办法。\n3. 第三条：城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n4. 第五条：重点地区的控制性详细规划未体现城市设计内容和要求的，应当及时修改完善。\n5. 第十五条：单体建筑设计和景观、市政工程方案设计应当符合城市设计要求。\n6. 第二十条：城市、县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况。\n7. 第二十一条：城市、县人民政府城乡规划主管部门开展城市规划实施评估时，应当同时评估城市设计工作实施情况。\n8. 第二十二条：城市设计的技术管理规定由国务院城乡规划主管部门另行制定。\n9. 第二十三条：各地可根据本办法，按照实际情况，制定实施细则和技术导则。\n10. 第二十四条：县人民政府所在地以外的镇可以参照本办法开展城市设计工作。\n11. 第二十五条：本办法自2017年6月1日起施行。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 城市设计管理办法的内容包括：\n\n1. 第一条：为提高城市建设水平，塑造城市风貌特色，推进城市设计工作，完善城市规划建设管理，依据《中华人民共和国城乡规划法》等法律法规，制定本办法。\n2. 第二条：城市、县人民政府所在地建制镇开展城市设计管理工作，适用本办法。\n3. 第三条：城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n4. 第五条：重点地区的控制性详细规划未体现城市设计内容和要求的，应当及时修改完善。\n5. 第十五条：单体建筑设计和景观、市政工程方案设计应当符合城市设计要求。\n6. 第二十条：城市、县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况。\n7. 第二十一条：城市、县人民政府城乡规划主管部门开展城

## 4.3 使用few shot example 来引导子query分解

In [23]:
data_map = {
    "电动汽车的品牌有哪些？各有什么特点？": [
        "当前市场上的主要电动汽车品牌。",
        "每个品牌的电动汽车品牌的基本技术规格，如续航里程、充电速度等。",
        "每个品牌的电动汽车型号，包括轿车、SUV、卡车等。",
        "每个品牌的充电基础设施和网络覆盖情况。",
        "每个品牌的电池技术和电动驱动系统。",
    ],
    "智能家居科技趋势及其在提高生活效率中的应用": ["当前最新的智能家居科技趋势是什么", "智能家居科技趋势的工作原理和特性", "哪些公司或产品正在引领这些智能家居科技趋势"],
    "人工智能技术在医疗诊断中的应用案例和效果分析": [
        "人工智能技术在医疗诊断领域中的主要应用案例",
        "收集每个应用案例中使用的具体人工智能技术，如深度学习、机器学习等",
        "分析这些应用案例中人工智能技术对诊断准确性的影响。",
        "研究人工智能技术在医疗诊断中的效果，特别关注是否提高了患者的治疗成功率。",
    ],
}

In [24]:
few_shot_examples = []
for key, values in data_map.items():
    meta_data = {}
    for i in range(len(values)):
        meta_data[f"sub_query_{i+1}"] = values[i]
    doc = Document(page_content=key, metadata={"sub_queries": meta_data})
    few_shot_examples.append(doc)

In [25]:
few_shot_retriever = FAISS.from_documents(few_shot_examples, embeddings)
few_shot_search = LangChainRetrievalTool(few_shot_retriever)

In [27]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    few_shot_retriever=few_shot_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [28]:
query = "住房和城乡建设部发布城市设计管理办法"
response = await agent.run(query)
response

AgentResponse(text='很抱歉，根据您提供的信息，我无法直接回答“住房和城乡建设部发布城市设计管理办法”的问题。如果您想了解更多关于该《城市设计管理办法》的信息，建议您查阅相关法律法规和官方文件。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 根据《城市设计管理办法》第一条，制定本办法的背景和原因是为了提高城市建设水平，塑造城市风貌特色，推进城市设计工作，完善城市规划建设管理，依据《中华人民共和国城乡规划法》等法律法规。\n\n    第2个段落: 该管理办法的主要内容和条款有：\n1. 第一章 总 则：规定了城市管理执法工作的目的、执法和服务水平的提高、城市管理秩序的维护、公民法人和其他组织的合法权益的保护等。\n2. 第二条：明确规定了城市管理执法的适用范围和定义，即城市、县人民政府所在地镇建成区内的城市管理执法活动以及执法监督活动，以及城市管理执法主管部门在城市管理领域根据法律法规规章规定履行行政处罚、行政强制等行政执法职责的行为。\n3. 第三十条：城市管理执法主管部门不得对罚款、没收违法所得设定任务和目标，罚款、没收违法所得的款项，应当按照规定全额上缴。\n4. 第三十一条：城市管理执法主管部门应当确定法制审核机构，配备一定比例符合条件的法制审核人员，对重大执法决定在执法主体、管辖权限、执法程序、事实认定、法律适用等方面进行法制审核。\n5. 第三十二条：城市管理执法主管部门开展执法活动，应当使用统一格式的行政执法文书。\n6. 第三十三条：行政执法文书的送达，依照民事诉讼法等法律规定执行。\n7. 第三十四条：城市管理执法主管部门应当通过门户网站、办事窗口等渠道或者场所，公开行政执法职责、权限、依据、监督方式等行政执法信息。\n8. 第六章 协作与配合：第三十五条、第三十六条和第三十七条分别规定了城市管理执法主管部门应当与有关部门建立行政执法信息互通共享机制，及时通报行政执法信息和相关行政管理信息；实行网格化管理；以及在执法活动中发现依法应当由其他部门查处的违法行为，应当及时告知或者移送有关部门。\n9. 第七章 执法监督：未提供具体条款内容。\n\n    第3个段落: 该管理办法的实施和影响是：\n1. 规范城市管理执法工作，提高执法

## 4.4 使用先验背景信息来引导子query分解

In [33]:
background_data = [
    "《城市管理执法办法》主要规定了城市管理执法的范围、主管部门、执法人员资格与行为准则、执法措施与程序，以及监督与责任追究机制，旨在确保城市管理执法活动规范、公正、文明，维护城市公共秩序和市容环境。",
    "《城市设计管理办法》主要内容是规定了城市设计的编制、审批、实施和管理的相关要求，旨在通过科学、合理的城市设计，塑造城市特色风貌，提升城市环境质量，促进城市可持续发展。该办法强调了城市设计的综合性、整体性和长远性，明确了各级政府和相关部门在城市设计中的职责和作用，为城市设计工作的规范化、制度化提供了保障。",
]

In [34]:
background_info = []
for item in background_data:
    meta_data = {}
    doc = Document(page_content=item)
    background_info.append(doc)

In [35]:
context_retriever = FAISS.from_documents(background_info, embeddings)
context_search = LangChainRetrievalTool(context_retriever)

In [36]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    context_retriever=context_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [37]:
query = "住房和城乡建设部发布城市设计管理办法"
response = await agent.run(query)
response

AgentResponse(text='很抱歉，文中没有提及住房和城乡建设部发布城市设计管理办法。\n\n城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n\n重点地区的控制性详细规划未体现城市设计内容和要求的，应当及时修改完善。\n\n单体建筑设计和景观、 市政工程方案设计应当符合城市设计要求。\n\n城市、 县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况。\n\n城市、 县人民政府城乡规划主管部门开展城市规划实施评估时，应当同时评估城市设计工作实施情况。\n\n城市设计的技术管理规定由国务院城乡规划主管部门另行制定。\n\n各地可根据本办法，按照实际情况，制定实施细则和技术导则。\n\n县人民政府所在地以外的镇可以参照本办法开展城市设计工作。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n\n重点地区的控制性详细规划未体现城市设计内容和要求的，应当及时修改完善。\n\n单体建筑设计和景观、 市政工程方案设计应当符合城市设计要求。\n\n城市、 县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况。\n\n城市、 县人民政府城乡规划主管部门开展城市规划实施评估时，应当同时评估城市设计工作实施情况。\n\n城市设计的技术管理规定由国务院城乡规划主管部门另行制定。\n\n各地可根据本办法，按照实际情况，制定实施细则和技术导则。\n\n县人民政府所在地以外的镇可以参照本办法开展城市设计工作。\n\n    第2个段落: 重点地区城市设计应当塑造城市风貌特色， 注重与山水自然的共生关系，协调市政工程，组织城市公共空间功能，注重建筑空间尺度，提出建筑高度、体量、风格、色彩等控制要求。\n\n检索语句: 住房和城乡建设部发布城市设计管理办法\n请根据以上检索结果回答检索语句的问题', token_count: 262>, <AIMessage role: 'assistant', content: '很抱歉，文中没有提及住房和城乡建设部发布城市设计管理